### Pregunta 1

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time

In [57]:
domains = {
    "Argentina": ".ar",
    "Belize": ".bz",
    "Bolivia": ".bo",
    "Brazil": ".br",
    "Chile": ".cl",
    "Colombia": ".co",
    "Costa Rica": ".cr",
    "Cuba": ".cu",
    "Dominica": ".dm",
    "Dominican Republic": ".do",
    "Ecuador": ".ec",
    "El Salvador": ".sv",
    "Grenada": ".gd",
    "Guatemala": ".gt",
    "Guyana": ".gy",
    "Haiti": ".ht",
    "Honduras": ".hn",
    "Jamaica": ".jm",
    "Mexico": ".mx",
    "Nicaragua": ".ni",
    "Panama": ".pa",
    "Paraguay": ".py",
    "Peru": ".pe",
    "Puerto Rico": ".pr",
    "Saint Kitts and Nevis": ".kn",
    "Saint Lucia": ".lc",
    "Saint Vincent and the Grenadines": ".vc",
    "Suriname": ".sr",
    "Trinidad and Tobago": ".tt",
    "Uruguay": ".uy",
    "Venezuela": ".ve",
    "British Virgin Islands": ".vg",
    "US Virgin Islands": ".vi"
}

In [17]:
driver = webdriver.Chrome()
url = 'https://ht.linkedin.com/jobs/data-scientist-empleos?position=1&pageNum=0'
driver.get(url)

while True:
    try:
        show_more_button = driver.find_elements(By.XPATH, '//*[@id="main-content"]/section[2]/button')
        if show_more_button and show_more_button[0].is_displayed():
            show_more_button[0].click()
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="main-content"]/section[2]/ul')))
            show_more_button = None
        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="main-content"]/section[2]/ul')))
    except:
        break

In [18]:
html = driver.page_source

In [19]:
soup = bs(html)
joblist = soup.select_one('#main-content > section.two-pane-serp-page__results-list > ul')
jobs = joblist.find_all('li')

hrefs = []
for j in jobs:
    content = j.find('a', class_='base-card__full-link')
    if content:
        hrefs.append(content['href'])

for i in hrefs:
    print(i)

https://ht.linkedin.com/jobs/view/senior-frontend-engineer-at-early-stage-startup-at-recruiting-from-scratch-3541530449?refId=Dk06p2jDMW1Dal%2BfhO836Q%3D%3D&trackingId=HA5Bxwf9l3CjNzH0NlgAgA%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card
https://ht.linkedin.com/jobs/view/senior-backend-engineer-ai-at-recruiting-from-scratch-3563606273?refId=Dk06p2jDMW1Dal%2BfhO836Q%3D%3D&trackingId=5mJM0fStbF4eV%2BpwSxqKXg%3D%3D&position=2&pageNum=0&trk=public_jobs_jserp-result_search-card
https://ht.linkedin.com/jobs/view/tech-lead-software-engineer-autonomous-car-startup-at-recruiting-from-scratch-3565088342?refId=Dk06p2jDMW1Dal%2BfhO836Q%3D%3D&trackingId=1SRKH1ixG2AMmdNNsoBdFg%3D%3D&position=3&pageNum=0&trk=public_jobs_jserp-result_search-card
https://ht.linkedin.com/jobs/view/senior-backend-engineer-data-at-recruiting-from-scratch-3541534243?refId=Dk06p2jDMW1Dal%2BfhO836Q%3D%3D&trackingId=I0AfXpj7l7TAw9sPeQoBjA%3D%3D&position=4&pageNum=0&trk=public_jobs_jserp-result_search-card

In [7]:
def scrape_company(url):
    options = Options()
    options.add_argument('--headless')
    
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    soup = bs(driver.page_source)

    #mail_box = driver.find_element(By.CSS_SELECTOR,"email-or-phone")
    #pass_box = driver.find_element(By.CSS_SELECTOR, "password")

    #mail_box.send_keys('a05785717@gmail.com')
    #pass_box.send_keys('qscwdb1!')

    #time.sleep(5)
    driver.close()

    return soup

In [20]:
df = pd.DataFrame(columns=['Title', 'Jornada', 'Ciudad', 'URL', 'Descripcion', 'Compañia', 'Nº Empleados', 'Rubro', 'País'])

c = 0
for url in hrefs:
    html = requests.get(url).content
    soup = bs(html)

    # TITLE
    title = soup.select_one('#main-content > section.core-rail.mx-auto.papabear\:w-core-rail-width.mamabear\:max-w-\[790px\].babybear\:max-w-\[790px\] > div > section.top-card-layout.container-lined.overflow-hidden.babybear\:rounded-\[0px\] > div > div.top-card-layout__entity-info-container.flex.flex-wrap.papabear\:flex-nowrap > div > h1')
    if title:
        title = title.text.strip()

    # JORNADA
    jornada = soup.select_one('#main-content > section.core-rail.mx-auto.papabear\:w-core-rail-width.mamabear\:max-w-\[790px\].babybear\:max-w-\[790px\] > div > div > section.core-section-container.my-3.description > div > ul > li:nth-child(2) > span')
    if jornada:
        jornada = jornada.text.strip()

    # CIUDAD
    city = soup.select_one('#main-content > section.core-rail.mx-auto.papabear\:w-core-rail-width.mamabear\:max-w-\[790px\].babybear\:max-w-\[790px\] > div > section.top-card-layout.container-lined.overflow-hidden.babybear\:rounded-\[0px\] > div > div.top-card-layout__entity-info-container.flex.flex-wrap.papabear\:flex-nowrap > div > h4 > div:nth-child(1) > span.topcard__flavor.topcard__flavor--bullet')
    if city: 
        city = city.text.strip()
    
    # DESCRIPCION
    descr = soup.select_one('#main-content > section.core-rail.mx-auto.papabear\:w-core-rail-width.mamabear\:max-w-\[790px\].babybear\:max-w-\[790px\] > div > div > section.core-section-container.my-3.description > div > div > section > div')
    if descr:
        descr = descr.get_text(' ').strip()

    # COMPAÑIA
    company = soup.select_one('#main-content > section.core-rail.mx-auto.papabear\:w-core-rail-width.mamabear\:max-w-\[790px\].babybear\:max-w-\[790px\] > div > section.top-card-layout.container-lined.overflow-hidden.babybear\:rounded-\[0px\] > div > div.top-card-layout__entity-info-container.flex.flex-wrap.papabear\:flex-nowrap > div > h4 > div:nth-child(1) > span:nth-child(1) > a')
    if company:
        urlC = company['href']
        soupC = scrape_company(urlC)

        company = soupC.select_one('#main-content > section.core-rail.mx-auto.papabear\:w-core-rail-width.mamabear\:max-w-\[790px\].babybear\:max-w-\[790px\] > section > div > div.top-card-layout__entity-info-container.flex.flex-wrap.papabear\:flex-nowrap > div:nth-child(1) > h1')
        if company:
            company = company.text.strip()
        
        # EMPLEADOS
        empleados = soupC.select_one('#main-content > section.core-rail.mx-auto.papabear\:w-core-rail-width.mamabear\:max-w-\[790px\].babybear\:max-w-\[790px\] > section > div > div.top-card-layout__entity-info-container.flex.flex-wrap.papabear\:flex-nowrap > div.top-card-layout__entity-info.flex-grow.flex-shrink-0.basis-0.babybear\:flex-none.babybear\:w-full.top-card-layout__entity-info--right-column.ml-details-container-padding.max-w-\[288px\].babybear\:my-2.babybear\:ml-0 > ul > li > div > a')
        if empleados:
            empleados = empleados.text.strip()
            empleados = ''.join([x for x in empleados if x.isdigit()])
            empleados = int(empleados)

        # RUBRO
        rubro = soupC.select_one('#main-content > section.core-rail.mx-auto.papabear\:w-core-rail-width.mamabear\:max-w-\[790px\].babybear\:max-w-\[790px\] > section > div > div.top-card-layout__entity-info-container.flex.flex-wrap.papabear\:flex-nowrap > div:nth-child(1) > h2')
        if rubro:
            rubro = rubro.text.strip()
        
        # PAIS
        pais = soupC.select_one('#main-content > section.core-rail.mx-auto.papabear\:w-core-rail-width.mamabear\:max-w-\[790px\].babybear\:max-w-\[790px\] > section > div > div.top-card-layout__entity-info-container.flex.flex-wrap.papabear\:flex-nowrap > div:nth-child(1) > h3')
        if pais:
            pais = pais.stripped_strings
            pais = [x.strip() for x in pais][0]
    else:
        company, empleados, rubro, pais = None, None, None, None

    # ADD TO DF
    row = {'Title': title, 'Jornada': jornada, 'Ciudad': city, 'URL': url, 'Descripcion': descr, 'Compañia': company, 'Nº Empleados': empleados, 'Rubro': rubro, 'País': pais}
    df = df.append(row, ignore_index=True)

    c += 1
    if c == 1000:
        break

df.to_csv('haiti.csv')